## Introduction
Analytical solutions are a powerful method to understand basic mechanisms of transport in the subsurface. However, modeling transport in more complex systems requires the use of numerical models. FloPy is a Python library that allows us to utilize the widely used USGS MODFLOW simulation programs. MODFLOW is the same modeling program you were introduced to in GEOSCI/GEOENG 627, except in this class we will use FloPy to run and process the results rather than a GUI. 

Why use FloPy?
* It's free!
* It is powerful. For example, Flopy can generate input files for the MODFLOW family programs in a few lines of code!
* It is flexible. You can solve an incredible array of problems by creating custom models, performing iterative modeling approaches or sensitivity analysis, and you can easily integrate data analysis workflows. 

In addition to the MODFLOW programs you worked with in the past, we will also be using the associated MT3D groundwater solute transport simulator for MODFLOW that supports simulation of transport using the MODFLOW flow solution.


### Make sure you have everything installed 
Prior to completing this notebook you need to have FloPy installed. Instructions can be [found here](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/MODFLOW%2C%20Python%2C%20and%20FloPy%20Setup.ipynb).

To check, run the following:

In [ ]:
import sys
# Import flopy
import flopy
'flopy' in sys.modules #True

## The first model
Once FloPy has been correctly installed we can set up our first MODFLOW 6 model!

In [ ]:
# Import a few libraries
import sys
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
# import pathlib
# Import the flopy library
import flopy
'flopy' in sys.modules #True

First find where you have your MODFLOW6 executables located on your system.

In [ ]:
# MODFLOW6 executable (only one)
# Executable location of Mf6.exe
# exe_path = "C:\\Hydro\\"
exe_path = "C:\\Hydro\\mf6.4.2\\mf6.4.2_win64\\bin\\"

exe_loc = os.path.dirname(exe_path)
print("Path to MODFLOW 6 executable:", exe_loc)

Let's use the same directory to save the data as the FloPy introduction and then create a path to this workspace. It may be useful to understand your current working directory, this should be whereever you have this notebook saved. You can double check this with the command 'os.getcwd()'.

In [ ]:
# This should return a path to your current working directory
current_directory = os.getcwd()
print(current_directory)

If this is not where you want to save stuff then uncomment the cell below and define the path to establish a new folder and set this to be the new working directory.

In [ ]:
# # define path
# path = pathlib.Path('C:\\Users\\zahas\\Dropbox\\Teaching\\Contaminant hydro 629\\Notebooks_unpublished')
# # if folder doesn't exist then make it 
# path.mkdir(parents=True, exist_ok=True)
# # set working directory to this new folder
# os.chdir(path)
# current_directory = os.getcwd()
# print(current_directory)

In [ ]:
# directory to save data
dirname = 'hello_model'
# Let's add that to the path of the current directory
workdir = os.path.join('.', dirname)

# if the path exists then we will move on, if not then create a folder with the 'directory_name'
if os.path.isdir(workdir) is False:
    os.mkdir(workdir) 
    print("Directory '% s' created" % workdir) 
else:
    print("Directory '% s' already exists" % workdir) 

# Model workspace and new sub-directory
model_ws = os.path.join(workdir, dirname)
print(model_ws)

Notice however that we don't yet name the folder where we will save data 'dirname'. This will be an input to our model function.


## 1D Model set up
The first thing we do is set the space and time units of our model and then define the model geometry. This is identical to the process that we used with the MODFLOW GUI program in GEOSCI/GEOENG 627.
    

In [ ]:
# name our model, for simplicity maybe use some formating like this
gwfname = 'gwf-' + dirname
# create the MF6 simulation
sim = flopy.mf6.MFSimulation(sim_name=dirname, exe_name=os.path.join(exe_loc, 'mf6.exe'), sim_ws=model_ws, 
                                verbosity_level = 2)
    
# time and length units - use lab units for now
length_units = "CENTIMETERS"
time_units = "MINUTES"

# Modflow stress periods
perlen = [30, 60*3] # min
# number of stress periods (MF input), calculated from period length input
nper = len(perlen)
# nstp (integer) is the number of time steps in a stress period.
# tsmult (double) is the multiplier for the length of successive time steps. 
tsmult = 1
tdis_rc = []
# loop through perlen and assign period lengths
for i in range(nper):
    tdis_rc.append((perlen[i], perlen[i]/10, tsmult))
flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units)

# Instantiating MODFLOW 6 groundwater flow model
gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, save_flows=True, 
        model_nam_file=f"{gwfname}.nam") 

# Instantiating MODFLOW 6 solver for flow model
imsgwf = flopy.mf6.ModflowIms(sim, complexity = "SIMPLE")
sim.register_ims_package(imsgwf, [gwf.name])

# Model information 
nlay = 1 # number of layers
nrow = 1 # number of rows
ncol = 101 # number of columns
top = 0 # grid size in direction of Lz
delc = 4.4 # grid size in direction of Ly, this was choosen such that the model has the same cross-sectional area as the column from the dispersion notebook example
delr = 0.1 # grid size in direction of Lx
botm  = -4.4
    
# length of model in selected units 
Lx = (ncol - 1) * delr
print("Model length is: " + str(Lx + delr) + ' ' + str(length_units))

# Instantiating MODFLOW 6 discretization package
flopy.mf6.ModflowGwfdis(gwf, length_units=length_units,
        nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc,
        top=top, botm=botm,
        filename=f"{gwfname}.dis")
    
# hydraulic conductivity
HK = 1. # what are the units here?

# Instantiating MODFLOW 6 node-property flow package
flopy.mf6.ModflowGwfnpf(gwf, save_flows=False, icelltype=0,
        k=HK) # k is the hydraulic conductivity 

## Define our initial flow conditions

In [ ]:
# Instantiating MODFLOW 6 initial conditions package for flow model
flopy.mf6.ModflowGwfic(gwf, strt=0.0)

## Define our flow boundary conditions
This is the input required for the MODFLOW basic package class. Boundary conditions are being assigned in the MODFLOW constant head package and therefore we are restricted to specifying head conditions. More rigorous constraint of constant flux boundaries requires the Flow and Head Boundary Package, the Well Package, or the Recharge Package.

In [ ]:
# advection velocity
v = 0.1
# porosity
prsity = 0.3
# discharge is based on advection velocity input (again in selected units)
q = v * prsity
print("Discharge = " + str(q))
# if want to achieve the desired flow rate in the model then use Darcy's law to 
# solve for necessary constant head boundary
h1 = q * Lx / HK
print("calculated head differential across column based on provided advection velocity = " + str(h1) )

# Constant head cells are specified on both ends of the model
chdspd = [[(0, 0, 0), h1], [(0, 0, ncol - 1), 0.0]]
# Instantiating MODFLOW 6 constant head package
flopy.mf6.ModflowGwfchd(gwf, maxbound=len(chdspd), 
                        stress_period_data=chdspd,
                        save_flows=False, pname="CHD1")

# FLow output control
flopy.mf6.ModflowGwfoc(gwf,
    head_filerecord=f"{gwfname}.hds",
    saverecord=[("HEAD", "ALL")],
    printrecord=[("HEAD", "FIRST"), ("HEAD", "LAST"),])

In [ ]:
#############################################################
############### NOW BUILD TRANSPORT #########################
print(f"Building mf6gwt model in...{model_ws}")
gwtname = "gwt_" + dirname
gwt = flopy.mf6.MFModel(sim,
        model_type="gwt6", modelname=gwtname,
        model_nam_file=f"{gwtname}.nam")
# gwt.name_file.save_flows = True

# create iterative model solution and register the gwt model with it
imsgwt = flopy.mf6.ModflowIms(sim, print_option="SUMMARY", linear_acceleration="BICGSTAB",
        filename=f"{gwtname}.ims")
sim.register_ims_package(imsgwt, [gwt.name])

# Instantiating MODFLOW 6 transport discretization package
flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
        delr=delr, delc=delc, top=top, botm=botm,
        filename=f"{gwtname}.dis")

# Initial conditions
# initial concentration set to zero everywhere
Ci=0
# Instantiating MODFLOW 6 transport initial concentrations
flopy.mf6.ModflowGwtic(gwt, strt=Ci, filename=f"{gwtname}.ic")

# Solute boundary conditions
Cinj = 1.5
cncspd = {0: [[(0, 0, 0), Cinj]], 1: [[(0, 0, 0), 0]]} # pulse
# Instantiating MODFLOW 6 transport constant concentration package
flopy.mf6.ModflowGwtcnc(gwt, maxbound=len(cncspd), stress_period_data=cncspd)

flopy.mf6.ModflowGwtssm(gwt)
    
# Mobile Storage and Transfer (MST) Package of the GWT Model for MODFLOW 6 represents solute mass storage, sorption, and frst- or zero-order decay in MOBILE domain.
flopy.mf6.ModflowGwtmst(gwt, porosity=prsity) # without reactions

# Instantiating MODFLOW 6 transport advection package
scheme = "TVD"
flopy.mf6.ModflowGwtadv(gwt, scheme=scheme)

## Dispersivity
Set the longitudinal dispersivity. Note that you can also set transverse dispersivity but that input would be ignored in this 1D model.

In [ ]:
al = 0.3 # longitudinal dispersivity
# define dispersion/diffusion behavior
flopy.mf6.ModflowGwtdsp(gwt, xt3d_off=True, alh=al, ath1=al)

In [ ]:
# Instantiating MODFLOW 6 transport output control package
flopy.mf6.ModflowGwtoc(gwt,
        budget_filerecord=f"{gwtname}.cbc",
        concentration_filerecord=f"{gwtname}.ucn",
        saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")],
        printrecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")])

# Instantiating MODFLOW 6 flow-transport exchange mechanism
flopy.mf6.ModflowGwfgwt(sim,
        exgtype="GWF6-GWT6",
        exgmnamea=gwfname, exgmnameb=gwtname,
        filename=f"{dirname}.gwfgwt")

# Write simulation
sim.write_simulation(silent=True)

Before running the model it is useful to show how we can go back and look at data from different packages. For example, we can retrive our dispersion information by printing the model.package, i.e `gwt.disp` as illustrated in the following cell.

More information on accessing simulation settings, models, and packages can be found [here](https://flopy.readthedocs.io/en/latest/Notebooks/mf6_data_tutorial01.html).

In [ ]:
# Print dispersion package
print(gwt.dsp)

We can also check what packages that we have implemented with the following

In [ ]:
package_list = gwt.get_package_list()
print(package_list)

## Run the simulation

In [ ]:
success, buff = sim.run_simulation(silent=False)

## Extract the simulation output 
In this case we are only extracting the steady state head field and the concentration data at each timestep. Depending on the output control and the use other observation utilities, your model may have more output.

In [ ]:
times = gwt.output.concentration().get_times()
# Extract head and concentration field data
head = gwf.output.head().get_data()
conc = gwt.output.concentration().get_alldata() # get_data() retrieves only the last timestep

In [ ]:
# Extract model grid
print(gwt.dis)

## Activity:
Interpret the following plots and think through the associated questions.

Note that the head and concentration values are in a 4D matrix with the following (t, z, y, x). Label the axes of each plot.

In [ ]:
# Replicate model grid
x = np.arange(0.1, 101*0.1, 0.1)
print(x.shape)

In [ ]:
# print dimensions of head array
print(head.shape)

# plot
plt.figure()
plt.plot(x, np.squeeze(head))
print(h1)

What are the axes of this plot. Can you explain what is happening within the model?

Now instead of focusing on grid cell head values, lets look at concentration values from the model. It might be helpful to first print the size of the concentration matrix to remind us what the axes of the array correspond to.

In [ ]:
print(conc.shape)

In [ ]:
plt.figure()
# now loop through the times and extract concentration profiles
for t in range(0, len(times)):
    cprof = conc[t, 0, 0, :]
    plt.plot(x, cprof)


What are the axes of this plot? Can you explain what is happening? From the plots do these look like a continous injection/source or pulse/finite length plume? How would the results change if we increase advection velocity. What happens if dispersivity is increased? 


Now plot the concentration data a little different. First, remember the rules of matrix indexing so that you follow what values are being extracted to plot the model reults.

In [ ]:
C_array = conc[:, 0, 0, -1]
plt.plot(times, C_array)
plt.show()

See if you can instead plot a breakthrough curve at a different location in the model. Can you explain the differences in the breakthrough curves at different locations? 